# Validation notebook
Here the FIRES implementation for multiclass and regression is validated and compared to other online features selcetion algorithms.


In [1]:
# import the functions needed fot validate and comparsion

import numpy as np
import cupy as cp
import pandas as pd
from timeit import default_timer as timer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from skmultiflow.data import FileStream
from skmultiflow.neural_networks import PerceptronMask

# using plotly for plots
#import plotly.graph_objects as go
import plotly.express as px



In [2]:
# paint mnist
def paint_digit(digit_values):
    fig = px.imshow(digit_values.reshape(28,28), binary_string=True)
    fig.update_layout(coloraxis_showscale=False)
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    fig.show()


In [15]:
# stability measurment as proposed in "Measurment the Stability of Feature Selection"

def pearson_stability_ij(arr1,arr2):
    d = len(arr1)
    k_i = np.sum(arr1)
    k_j = np.sum(arr2)
    x_hat_i = k_i / d
    x_hat_j = k_j / d
    arr1 = arr1 - x_hat_i
    arr2 = arr2 - x_hat_j
    dividend = 1/d * np.sum(arr1*arr2)
    divisor = np.sqrt(1/d*np.sum(arr1**2))*np.sqrt(1/d*np.sum(arr2**2))
    return dividend/divisor

def stability_factor(selected_ftrs):
   M = len(selected_ftrs)
   sum_stabilities = 0
   for i in range(M):
       for j in range(i+1, M):
           sum_stabilities += pearson_stability_ij(selected_ftrs[i], selected_ftrs[j])
   return 1/(M*(M-1))*sum_stabilities * 2   

In [6]:
# import algorithms
from fires import FIRES
from fires_cuda import FIRES as FC
from ofs import OFS, MC_OFS
from ofssgr import OFSSGD, MC_OFSSGD
from fsds import StreamFeatWeight

## Multiclass Data

Here the FIRES softmax implementation is compared to the FSDS, OFS and OFSSGD oun multiclass data.


### Load Datasets as Streaming Data

In [7]:
# MNIST data
stream = FileStream('datasets/Multiclass/mnist_train_normalized.csv', target_idx=0)
stream.prepare_for_use()
dataset_name = "mnist"

# load test data
test_data = pd.read_csv('datasets/Multiclass/mnist_test_normalized.csv')
test_y = test_data[0].to_numpy()
test_x = test_data.drop(columns="0").to_numpy()


In [ ]:
# Human Activity Recognition
# labels changed from [1,...,6] to [0,...,5]
# rows shuffled
# split into train set with 7352 instances and test set with 2948
stream = FileStream('datasets/Multiclass/har_train.csv', target_idx = 651)
stream.prepare_for_use()
dataset_name = "har"

# load test data
test_data = pd.read_csv('datasets/Multiclass/har_csv.csv')
test_y = test_data["Class"].to_numpy()
test_x = test_data.drop(columns="Class").to_numpy()


### Test Perceptron
the same model is used for all algorithms


In [8]:
predictor = PerceptronMask()
x,y = stream.next_sample(batch_size=100)
predictor.partial_fit(x,y, stream.target_values)

PerceptronMask(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
               fit_intercept=True, max_iter=1000, n_iter_no_change=5,
               n_jobs=None, penalty=None, random_state=0, shuffle=True,
               tol=0.001, validation_fraction=0.1, verbose=0, warm_start=False)

### Test without feature selection


In [12]:
accuarcy_pure = []
while stream.has_more_samples():
    x, y = stream.next_sample(batch_size=100)
    y_pred = predictor.predict(x)
    accuarcy_pure.append(accuracy_score(y, y_pred))
    predictor.partial_fit(x,y)

    


In [14]:
fig = px.line(y = accuarcy_pure, title="Accuracy without ftr selection", labels={"x":"batches", "y":"accuracy"})
fig.show()

In [ ]:
y_pred = predictor.predict(test_x)
accuracy = accuracy_score(test_x, y_pred)
print("For the test dataset the previous trained predictor reached: {}".format(accuracy))

In [15]:
stream.restart()

### FIRES Framework

In [9]:
fires_model = FC(n_total_ftr=stream.n_features,
                    target_values=stream.target_values,
                    mu_init=0,
                    sigma_init=1,
                    model='softmax')
#print(fires_model.n_mc_samples)

#n_selected_ftr = 100
#x, y = stream.next_sample(batch_size=100)
# Select features
#start_time = timer()
#ftr_weights = fires_model.weigh_features(x, y)  # Get feature weights with FIRES
#ftr_selection = np.argsort(ftr_weights)[::-1][:n_selected_ftr]
#print(timer()-start_time)                    

In [10]:
fires_cuda_accuracy = []
#fsds_f1 = []
fires_cuda_times = []
n_selected_ftr = 100

fires_cuda_selected_ftrs = []
fires_cuda_stability = []

start_time_all = timer()
while stream.has_more_samples():
    # Load a new sample
    x, y = stream.next_sample(batch_size=100)
    # Select features
    start_time = timer()
    ftr_weights = fires_model.weigh_features(x,y)
    ftr_selection = np.argsort(ftr_weights)[::-1][:n_selected_ftr]
    fires_cuda_times.append(timer()-start_time)

    # Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
    x_reduced = np.zeros(x.shape)
    x_reduced[:, ftr_selection] = x[:, ftr_selection]

    # stability test
    ftr_array = np.zeros(stream.n_features)
    ftr_array[ftr_selection] = 1
    fires_cuda_selected_ftrs.append(ftr_array)

    if len(fires_cuda_selected_ftrs) >= 10:
        stability = stability_factor(fires_cuda_selected_ftrs[-10:])
        fires_cuda_stability.append(stability)


    # Test
    y_pred = predictor.predict(x_reduced)
    
    fires_cuda_accuracy.append(accuracy_score(y, y_pred))
    #fsds_f1.append(f1_score(y, y_pred, average=None, labels=stream.target_values))


    # Train
    predictor.partial_fit(x_reduced, y)

# Restart the FileStream
end_time_all = timer()
fires_cuda_run_time = timer() - start_time_all
print("The whole fires_cuda run took {}".format(fires_cuda_run_time))
stream.restart()


38
1.0
39
1.0
4
45
1.0
46
1.0
47
1.0
48
1.0
49
1.0
5
56
1.0
57
1.0
58
1.0
59
1.0
6
67
1.0
68
1.0
69
1.0
7
78
1.0
79
1.0
8
89
1.0
9
0
01
1.0
02
1.0
03
1.0
04
1.0
05
1.0
06
1.0
07
1.0
08
1.0
09
1.0
1
12
1.0
13
1.0
14
1.0
15
1.0
16
1.0
17
1.0
18
1.0
19
1.0
2
23
1.0
24
1.0
25
1.0
26
1.0
27
1.0
28
1.0
29
1.0
3
34
1.0
35
1.0
36
1.0
37
1.0
38
1.0
39
1.0
4
45
1.0
46
1.0
47
1.0
48
1.0
49
1.0
5
56
1.0
57
1.0
58
1.0
59
1.0
6
67
1.0
68
1.0
69
1.0
7
78
1.0
79
1.0
8
89
1.0
9
0
01
1.0
02
1.0
03
1.0
04
1.0
05
1.0
06
1.0
07
1.0
08
1.0
09
0.9885380116959064
1
12
1.0
13
1.0
14
1.0
15
1.0
16
1.0
17
1.0
18
1.0
19
0.9885380116959064
2
23
1.0
24
1.0
25
1.0
26
1.0
27
1.0
28
1.0
29
0.9885380116959064
3
34
1.0
35
1.0
36
1.0
37
1.0
38
1.0
39
0.9885380116959064
4
45
1.0
46
1.0
47
1.0
48
1.0
49
0.9885380116959064
5
56
1.0
57
1.0
58
1.0
59
0.9885380116959064
6
67
1.0
68
1.0
69
0.9885380116959064
7
78
1.0
79
0.9885380116959064
8
89
0.9885380116959064
9
0
01
1.0
02
1.0
03
1.0
04
1.0
05
1.0
06
1.0
07
1.0
08
0.9885380

In [8]:
fig = px.line(y = fires_cuda_accuracy, title="Accuracy for Fires algorithm", labels={"x":"batches", "y":"accuracy"})
fig.show()

In [11]:
fig = px.line(y = fires_cuda_stability, title="Stability for Fires algorithm", labels={"x":"batches", "y":"accuracy"})
fig.show()

In [12]:
if dataset_name == "mnist":
    img = np.zeros((784))
    img[ftr_selection] = 1
    paint_digit(img)


In [ ]:
y_pred = predictor.predict(test_x)
accuracy = accuracy_score(test_x, y_pred)
print("For the test dataset the previous trained predictor reached: {}".format(accuracy))

### FIRES without cuda

Only one batch because of long calculation time

In [ ]:
stream.restart()
print("Choosen dataset: {}".format(dataset_name))
predictor = PerceptronMask()
x,y = stream.next_sample(batch_size=100)
predictor.partial_fit(x,y, stream.target_values)

In [ ]:
fires_model = FIRES(n_total_ftr=stream.n_features,
                    target_values=stream.target_values,
                    mu_init=0,
                    sigma_init=1,
                    model='softmax')
print(fires_model.n_mc_samples)

n_selected_ftr = 100
x, y = stream.next_sample(batch_size=100)
# Select features
start_time = timer()
ftr_weights = fires_model.weigh_features(x, y)  # Get feature weights with FIRES
ftr_selection = np.argsort(ftr_weights)[::-1][:n_selected_ftr]
print(timer()-start_time)     

### FSDS algorithm


In [13]:
stream.restart()
print("Choosen dataset: {}".format(dataset_name))
predictor = PerceptronMask()
x,y = stream.next_sample(batch_size=100)
predictor.partial_fit(x,y, stream.target_values)

Choosen dataset: mnist


PerceptronMask(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
               fit_intercept=True, max_iter=1000, n_iter_no_change=5,
               n_jobs=None, penalty=None, random_state=0, shuffle=True,
               tol=0.001, validation_fraction=0.1, verbose=0, warm_start=False)

In [14]:
fsds_model = StreamFeatWeight(m=stream.n_features, k=stream.n_classes)
fsds_model.low_rank_approximation(x.T) # needs some pretraining in the first run

n_selected_ftr = 100

fsds_selected_ftrs = []
fsds_stability = []

fsds_accuracy = []
#fsds_f1 = []
fsds_times = []

start_time_all = timer()
while stream.has_more_samples():
    # Load a new sample
    x, y = stream.next_sample(batch_size=100)
    # Select features
    start_time = timer()
    ftr_weights = fsds_model.low_rank_approximation(x.T)
    ftr_selection = np.argsort(ftr_weights)[::-1][:n_selected_ftr]
    fsds_times.append(timer()-start_time)

    # Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
    x_reduced = np.zeros(x.shape)
    x_reduced[:, ftr_selection] = x[:, ftr_selection]

     # stability test
    ftr_array = np.zeros(stream.n_features)
    ftr_array[ftr_selection] = 1
    fsds_selected_ftrs.append(ftr_array)

    if len(fsds_selected_ftrs) >= 10:
        stability = stability_factor(fsds_selected_ftrs[-10:])
        fsds_stability.append(stability)

    # Test
    y_pred = predictor.predict(x_reduced)
    
    fsds_accuracy.append(accuracy_score(y, y_pred))
    #fsds_f1.append(f1_score(y, y_pred, average=None, labels=stream.target_values))


    # Train
    predictor.partial_fit(x_reduced, y)

# Restart the FileStream
end_time_all = timer()
fsds_run_time = timer() - start_time_all
print("The whole fsds run took {}".format(fsds_run_time))
stream.restart()

64
15
1.0
16
0.9770760233918129
17
0.9885380116959064
18
0.9770760233918129
19
0.9885380116959064
2
23
0.9885380116959064
24
0.9885380116959064
25
1.0
26
0.9770760233918129
27
0.9885380116959064
28
0.9770760233918129
29
0.9885380116959064
3
34
1.0
35
0.9885380116959064
36
0.9656140350877193
37
0.9770760233918129
38
0.9656140350877193
39
0.9770760233918129
4
45
0.9885380116959064
46
0.9656140350877193
47
0.9770760233918129
48
0.9656140350877193
49
0.9770760233918129
5
56
0.9770760233918129
57
0.9885380116959064
58
0.9770760233918129
59
0.9885380116959064
6
67
0.9885380116959064
68
0.9885380116959064
69
0.9885380116959064
7
78
0.9885380116959064
79
1.0
8
89
0.9885380116959064
9
0
01
1.0
02
0.9885380116959064
03
0.9885380116959064
04
1.0
05
0.9770760233918129
06
0.9885380116959064
07
0.9770760233918129
08
0.9885380116959064
09
0.9885380116959064
1
12
0.9885380116959064
13
0.9885380116959064
14
1.0
15
0.9770760233918129
16
0.9885380116959064
17
0.9770760233918129
18
0.9885380116959064
19
0

In [16]:
fig = px.line(y = fsds_accuracy, title="Accuracy for FSDS algorithm", labels={"x":"batches", "y":"accuracy"})
fig.show()

In [17]:
fig = px.line(y = fsds_stability, title="Stability for FSDS algorithm", labels={"x":"batches", "y":"accuracy"})
fig.show()

In [19]:
if dataset_name == "mnist":
    img = np.zeros((784))
    img[ftr_selection] = 1
    paint_digit(img)


In [ ]:
y_pred = predictor.predict(test_x)
accuracy = accuracy_score(test_x, y_pred)
print("For the test dataset the previous trained predictor reached: {}".format(accuracy))

### OFS algorithm

In [44]:
stream.restart()
predictor = PerceptronMask()
x,y = stream.next_sample(batch_size=100)
predictor.partial_fit(x,y, stream.target_values)

PerceptronMask(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
               fit_intercept=True, max_iter=1000, n_iter_no_change=5,
               n_jobs=None, penalty=None, random_state=0, shuffle=True,
               tol=0.001, validation_fraction=0.1, verbose=0, warm_start=False)

In [45]:
n_selected_ftr = 100
ofs = MC_OFS(regularization_param = 0.01, step_size = 0.1, n_selected_ftr=n_selected_ftr, n_total_ftr=stream.n_num_features, n_classes=stream.n_classes)

ofs_accuracy = []
ofs_selected_ftrs = []
ofs_stability = []

start_time_all = timer()
while stream.has_more_samples():
    # Load a new sample
    x, y = stream.next_sample(batch_size=100)

    # Select features
    for idx, label in enumerate(y):
        ofs.train(x[idx],label)

    selected_ftr = ofs.get_feature_indices()
    # Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
    x_reduced = np.zeros(x.shape)
    x_reduced[:, selected_ftr] = x[:, selected_ftr]

     # stability test
    ftr_array = np.zeros(stream.n_features)
    ftr_array[selected_ftr] = 1
    ofs_selected_ftrs.append(ftr_array)

    if len(ofs_selected_ftrs) >= 10:
        stability = stability_factor(ofs_selected_ftrs[-10:])
        ofs_stability.append(stability)

    # Test
    y_pred = predictor.predict(x_reduced)
    ofs_accuracy.append(accuracy_score(y, y_pred))

    # Train
    predictor.partial_fit(x_reduced, y)

end_time_all = timer()
ofs_run_time = timer() - start_time_all
print("The whole ofs run took {}".format(ofs_run_time))

# Restart the FileStream
stream.restart()

The whole ofs run took 7.114204711999264


In [46]:
fig = px.line(y = ofs_accuracy, title="Accuracy for OFS algorithm", labels={"x":"batches", "y":"accuracy"})
fig.show()

In [47]:
fig = px.line(y = ofs_stability, title="Stability for OFS algorithm", labels={"x":"batches", "y":"stability"})
fig.show()

In [25]:
if dataset_name == "mnist":
    img = np.zeros((784))
    img[selected_ftr] = 1
    paint_digit(img)


In [ ]:
y_pred = predictor.predict(test_x)
accuracy = accuracy_score(test_x, y_pred)
print("For the test dataset the previous trained predictor reached: {}".format(accuracy))

### OFSSGR algorithm

In [38]:
stream.restart()
predictor = PerceptronMask()
x,y = stream.next_sample(batch_size=100)
predictor.partial_fit(x,y, stream.target_values)

PerceptronMask(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
               fit_intercept=True, max_iter=1000, n_iter_no_change=5,
               n_jobs=None, penalty=None, random_state=0, shuffle=True,
               tol=0.001, validation_fraction=0.1, verbose=0, warm_start=False)

In [39]:

ofssgd_model = MC_OFSSGD(reduction_threshold=0.4, reduction_value=0.1, regularization_param=0.01, step_size=0.2, n_total_ftrs=stream.n_num_features, n_classes=stream.n_classes)

ofssgd_accuracy = []
ofssgd_selected_ftrs = []
ofssgd_stability = []

start_time_all = timer()
while stream.has_more_samples():
    # Load a new sample
    x, y = stream.next_sample(batch_size=100)

    # Select features
    for idx, label in enumerate(y):
        ofssgd_model.train(x[idx],label)

    selected_ftr = ofssgd_model.get_feature_indices()
    # Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
    x_reduced = np.zeros(x.shape)
    x_reduced[:, selected_ftr] = x[:, selected_ftr]

    # stability test
    ftr_array = np.zeros(stream.n_features)
    ftr_array[selected_ftr] = 1
    ofssgd_selected_ftrs.append(ftr_array)

    if len(ofssgd_selected_ftrs) >= 10:
        stability = stability_factor(ofssgd_selected_ftrs[-10:])
        ofssgd_stability.append(stability)

    # Test
    y_pred = predictor.predict(x_reduced)
    ofssgd_accuracy.append(accuracy_score(y, y_pred))

    # Train
    predictor.partial_fit(x_reduced, y)

end_time_all = timer()
ofssgd_run_time = timer() - start_time_all
print("The whole ofssgd run took {}".format(ofssgd_run_time))

# Restart the FileStream
stream.restart()

333
316
284
316
339
292
316
344
348
The whole ofssgd run took 115.34339147999981


In [40]:
fig = px.line(y = ofssgd_accuracy, title="Accuracy for OFSSGD algorithm", labels={"x":"batches", "y":"accuracy"})
fig.show()

In [41]:
fig = px.line(y = ofssgd_stability, title="Stability for OFSSGD algorithm", labels={"x":"batches", "y":"stability"})
fig.show()

In [42]:
if dataset_name == "mnist":
    img = np.zeros((784))
    img[selected_ftr] = 1
    paint_digit(img)


In [32]:
y_pred = predictor.predict(test_x)
accuracy = accuracy_score(test_x, y_pred)
print("For the test dataset the previous trained predictor reached: {}".format(accuracy))

590